In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [46]:
URL = 'https://m.imdb.com/chart/toptv/'

In [62]:
Film_title_list = []
rating_list=[]
Release_year_list = []

Director_list = []
Stars_list = []


headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
page = requests.get(URL, headers = headers)
soup = BeautifulSoup(page.text , 'html.parser')

# Film Title
Film_titles = soup.findAll('div' ,{'class':"ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b189961a-9 iALATN cli-title"})
for Film_title in Film_titles:
  Film_title_list.append(Film_title.contents[0].find("h3").text)

# Rating
ratings = soup.findAll('svg' ,{'class':"ipc-icon ipc-icon--star-inline"})
for rating in ratings:
  rating_list.append(rating.find_next(text=True).strip())

# Release Year
release_years = soup.findAll('div' ,{'class':"sc-b189961a-7 feoqjK cli-title-metadata"})
for release_year in release_years:
  Release_year_list.append(release_year.contents[0].text.strip())

# Create Tv Shows Url List

tv_shows_url_list = []
urls = soup.findAll('a' , {'class':"ipc-lockup-overlay ipc-focusable"})
for i in range(250):
  tv_shows_url_list.append("https://m.imdb.com/" + urls[i]["href"])
print(tv_shows_url_list)



['https://m.imdb.com//title/tt0903747/?ref_=chttvtp_i_1', 'https://m.imdb.com//title/tt5491994/?ref_=chttvtp_i_2', 'https://m.imdb.com//title/tt0795176/?ref_=chttvtp_i_3', 'https://m.imdb.com//title/tt0185906/?ref_=chttvtp_i_4', 'https://m.imdb.com//title/tt7366338/?ref_=chttvtp_i_5', 'https://m.imdb.com//title/tt0306414/?ref_=chttvtp_i_6', 'https://m.imdb.com//title/tt0417299/?ref_=chttvtp_i_7', 'https://m.imdb.com//title/tt6769208/?ref_=chttvtp_i_8', 'https://m.imdb.com//title/tt0141842/?ref_=chttvtp_i_9', 'https://m.imdb.com//title/tt2395695/?ref_=chttvtp_i_10', 'https://m.imdb.com//title/tt0081846/?ref_=chttvtp_i_11', 'https://m.imdb.com//title/tt9253866/?ref_=chttvtp_i_12', 'https://m.imdb.com//title/tt0944947/?ref_=chttvtp_i_13', 'https://m.imdb.com//title/tt0071075/?ref_=chttvtp_i_14', 'https://m.imdb.com//title/tt7678620/?ref_=chttvtp_i_15', 'https://m.imdb.com//title/tt1355642/?ref_=chttvtp_i_16', 'https://m.imdb.com//title/tt2861424/?ref_=chttvtp_i_17', 'https://m.imdb.com//t

<ipython-input-62-9b1d1cb39ce8>:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  rating_list.append(rating.find_next(text=True).strip())


In [48]:
Movie_image = []
Genre_list = []
for url in tv_shows_url_list:
  time.sleep(1)
  headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
  page = requests.get(url, headers = headers)
  soup = BeautifulSoup(page.text , 'html.parser')

  # Movie Image
  image = soup.find('a' , {'class':"ipc-lockup-overlay ipc-focusable"})
  Movie_image.append("https://m.imdb.com/" + image["href"])

  # Genre
  genre_batch = soup.findAll('a' , {'class':"ipc-chip ipc-chip--on-baseAlt"})
  temp = []
  for genre in genre_batch:
    temp.append(genre.text)
  print(temp)
  Genre_list.append(temp)


['Crime', 'Drama', 'Thriller']
['Documentary']
['Documentary', 'Family']
['Drama', 'History', 'War']
['Drama', 'History', 'Thriller']
['Crime', 'Drama', 'Thriller']
['Animation', 'Action', 'Adventure']
['Documentary']
['Crime', 'Drama']
['Documentary']
['Documentary']
['Documentary']
['Action', 'Adventure', 'Drama']
['Documentary', 'History', 'War']
['Animation', 'Family']
['Animation', 'Action', 'Adventure']
['Animation', 'Adventure', 'Comedy']
['Documentary']
['Documentary', 'Biography', 'History']
['Drama', 'Fantasy', 'Horror']
['Crime', 'Drama', 'Mystery']
['Documentary', 'History', 'War']
['Animation', 'Action', 'Adventure']
['Animation', 'Action', 'Adventure']
['Comedy']
['Documentary']
['Animation', 'Action', 'Adventure']
['Biography', 'Crime', 'Drama']
['Crime', 'Drama']
['Documentary', 'Comedy', 'Reality-TV']
['Documentary']
['Documentary']
['Adventure', 'Drama', 'Sci-Fi']
['Animation', 'Action', 'Adventure']
['Comedy']
['Documentary', 'History', 'War']
['Animation', 'Crime', 

In [63]:
Final_df = pd.DataFrame({'Film Title' : Film_title_list,
                         'IMDb Rating' : rating_list,
                         'Release Date': Release_year_list,
                         'Genre': Genre_list,
                         'Movie Image': Movie_image})
Final_df

,Film Title,IMDb Rating,Release Date,Genre,Movie Image
0,1. Breaking Bad,9.5,2008–2013,"[Crime, Drama, Thriller]",https://m.imdb.com//title/tt0903747/mediaviewe...
1,2. Planet Earth II,9.5,2016,[Documentary],https://m.imdb.com//title/tt5491994/mediaviewe...
2,3. Planet Earth,9.4,2006,"[Documentary, Family]",https://m.imdb.com//title/tt0795176/mediaviewe...
3,4. Band of Brothers,9.4,2001,"[Drama, History, War]",https://m.imdb.com//title/tt0185906/mediaviewe...
4,5. Chernobyl,9.3,2019,"[Drama, History, Thriller]",https://m.imdb.com//title/tt7366338/mediaviewe...
...,...,...,...,...,...
245,246. Gintama,8.7,2005–2021,"[Animation, Action, Comedy]",https://m.imdb.com//title/tt0988818/mediaviewe...
246,247. Tear Along the Dotted Line,8.6,2021,"[Animation, Comedy, Drama]",https://m.imdb.com//title/tt15614372/mediaview...
247,248. Alchemy of Souls,8.7,2022–,"[Action, Drama, Fantasy]",https://m.imdb.com//title/tt20859920/mediaview...
248,249. Chef's Table,8.5,2015–2019,"[Documentary, Biography]",https://m.imdb.com//title/tt4295140/mediaviewe...


In [64]:
Final_df.to_excel('Data.xlsx')